In [1]:
%matplotlib inline
import numpy as np

from minefield import Minefield
from network import DQ_Sweeper

from tqdm import tqdm

rewards = {'safe_reward' : 15, 
          'repeat_loss' : -5, 
          'mine_loss' : -10, 
          'completion_reward' : 30}
env = Minefield(8, 8, 10, **rewards)

filters = [10, 8, 5]
sizes = [5, 5, 5]
env_shape = (8, 8, 11)

model = DQ_Sweeper(env, filters, sizes, 0.01, 1, 100, 0.2, f = 'model_June09')

In [2]:
#model.show_single(seed = 0)
for i in tqdm(range(50)):
    model.run_episode(use_epsilon = False)

100%|██████████| 50/50 [00:12<00:00,  3.88it/s]


In [3]:
model.fit(64)

In [4]:
batch_size = 64
n_hist = model.logger.states.shape[0]

samps = np.random.choice(n_hist, size = batch_size, replace = False)

states = model.logger.states[samps]
actions = [model.logger.actions[i] for i in samps]

rewards = np.array([model.logger.rewards[i] for i in samps])

is_end = np.array([model.logger.is_end[i] for i in samps])

preds = model.model.predict(states)

# mod to deal with case of last sample being chosen
q_maxes = model.model.predict(model.logger.states[(samps + 1) % n_hist]).max(axis = (1, 2, 3)) 
adj = q_maxes * (1 - is_end) # if not the final move, add value of next state
vals = rewards + adj

In [8]:
vals

array([ 33.2707119 ,  37.16631699,  36.16477585,  43.05189896,
        32.85065269,  33.37192345,  37.79580116,  36.27749634,
        37.17007828,  36.43553734,  32.62354851,  35.88439751,
        34.4118824 ,  33.41992378,  31.8886261 ,  37.76851654,
        35.72603035,  33.44233131,  33.10858727,  35.94563293,
        31.33359146,  36.94706154,  47.76549149,  34.23225403,
        35.77067947,  35.5647068 , -10.        ,  37.40695763,
        30.        ,  43.0510807 ,  33.03935432,  32.22664833,
        38.25855255,  35.4971447 ,  35.9268074 ,  35.96170616,
        34.79483032,  36.1694088 ,  36.10784721,  34.82097435,
        33.79294968,  36.24902916,  36.96309853,  30.39759064,
        29.95071793,  29.20947361,  37.11466599,  34.11537743,
        35.32587051,  30.13088703,  36.21368408,  37.95280457,
        34.53451347,  33.41639709,  33.85217857,  37.82502174,
        34.7365284 ,  33.10308456,  36.75537491,  35.08343315,
        33.79097176,  36.09489632,  46.47138214,  32.33

In [6]:
is_end

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [7]:
n_next = (1 - is_end).sum()

plt.figure(figsize = (10, 5 * n_next))
for i in range(n_next):
    l = plt.subplot(n_next, 2, 2 * i + 1)
    r = plt.subplot(n_next, 2, 2 * i + 2)
    
    

NameError: name 'plt' is not defined

In [ ]:
states[4][:, :, :-1]

In [ ]:
rewards[np.where(is_end)]